In [5]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/openai-chat-completions/infer"
)

completion = client.chat.completions.create(
    model="openai-chat-completions",
    messages=[
      {
        "role": "user",
        "content": "You are a helpful assistant."
      },
      {
      	"role": "assistant",
      	"content": "Hello! How can I help you?"
      },
      {
      	"role": "user",
      	"content": "What is the capital of Romania?"
      }
    ],
)
print(completion.choices[0].message.content)

APIConnectionError: Connection error.

In [10]:
import subprocess

def get_mesh_ip():
    cmd = f"kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{{.status.loadBalancer.ingress[0].ip}}'"
    return subprocess.check_output(cmd, shell=True).decode('utf-8')

In [13]:
get_mesh_ip()

'172.22.255.2'

In [12]:
import json
import pprint
import requests


inference_request = {
    "inputs": [
        {
            "name": "role", 
            "shape": [2], 
            "datatype": "BYTES", 
            "data": ["system", "user"]
        },
        {
            "name": "content",
            "shape": [2],
            "datatype": "BYTES",
            "data": [
                json.dumps(["You are a helpful assistant"]), 
                json.dumps(["Hello from MLServer!"]),
            ],
        },
        {
            "name": "type",
            "shape": [2],
            "datatype": "BYTES",
            "data": [
                json.dumps(["text"]), 
                json.dumps(["text"]),
            ]
        }
    ]
}

endpoint = f"http://{get_mesh_ip()}/v2/models/openai-chat-completions/infer"
response = requests.post(endpoint, json=inference_request)
pprint.pprint(response.json(), depth=4)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)